In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip the kaggle dataset and extract all the files. 
from zipfile import ZipFile 
file_name= '/content/drive/MyDrive/Dataset/sign-to-speech-dataset.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

In [ ]:
#install the libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from IPython.display import Image

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

import cv2
import os



In [ ]:
data_path='DATASET'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
print(categories)
print(labels)

In [ ]:
data_path='/content/DATASET/train'
classes_path=os.listdir(data_path)
print(classes_path)
labels_classes=[i for i in range(len(classes_path))]
print(labels_classes)

In [ ]:
data_path='/content/DATASET/train'
label_classes_dict = {}
for directory in classes_path:
  for value in labels_classes:
    label_classes_dict[directory] = value
    labels_classes.remove(value)
    break
label_classes_dict

In [ ]:
data_path='/content/DATASET/'
img_size=128
data=[]
target=[]
c=0
minValue = 70
for category in categories:
    
    cat_path=os.path.join(data_path,category)
    print(cat_path)
    cat_names=os.listdir(cat_path)
    print(cat_names)
    for classes in cat_names:
        folder_path=os.path.join(data_path,category,classes)
        print(folder_path)
        img_names=os.listdir(folder_path)
        #print(img_names)
        for img_name in img_names:
            #print(img_name)
            img_path=os.path.join(folder_path,img_name)
            img=cv2.imread(img_path)
            
            try:
                gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
                blur = cv2.GaussianBlur(gray,(5,5),2)
                th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
                ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                #res=np.array(res)
                #print(type(res))
            #Converting the image into gray scale
                resized=cv2.resize(res,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
                data.append(resized)
                #print(data)
                target.append(label_classes_dict[classes])
            except Exception as e:
                print('Exception:',e)

In [ ]:
data =np.array(data)
target=np.array(target)
(data.shape, target.shape)

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

np.save('data_img',data)
np.save('target',new_target)

data=np.load('data_img.npy')
target=np.load('target.npy')

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(data,new_target,
                                                                    test_size=0.2)

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=96, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=27, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
model.summary()

In [ ]:
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')
earlystopping = EarlyStopping(monitor = 'val_accuracy',
                              min_delta = 0.01,
                              patience = 5,
                              mode = 'auto',
                              restore_best_weights = True,
                              verbose = 1)
history = model.fit(train_data,
                    train_target,
                    shuffle=True,
                    epochs=20,
                    callbacks=[checkpoint, earlystopping],
                    validation_split=0.2)

In [ ]:
print('Validation Results')
print(model.evaluate(test_data,test_target))

In [ ]:
print("saving model...")
model.save('american-sign-language-detection-model.h5')
print("Done!!!")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend(['train_loss','val_loss'], loc=0)
plt.show()

In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend(['train_accuracy','val_accuracy'], loc=0)
plt.show()